# ***Harleen Kaur Bagga***

# The capital asset pricing model and the security market line

In [20]:
""" 
Linear regression with SciPy 
"""
from scipy import stats#Using the stats module of SciPy, we will perform a least squares regression on the CAPM model

stock_returns = [0.065, 0.0265, -0.0593, -0.001, 0.0346]
mkt_returns = [0.055, -0.09, -0.041, 0.045, 0.022]
beta, alpha, r_value, p_value, std_err = \
    stats.linregress(stock_returns, mkt_returns)#returns slope of regression line(beta) ,the correlation function(alpha)
    #the p-value for a hypothesis test with null hypothesis of a zero slope(p_value),the standard error of the estimate(std_err)



In [21]:
print(beta, alpha)#print slope and correlation function

0.5077431878770808 -0.008481900352462384


# Multivariate linear regression of factor models

In [22]:
""" 
Least squares regression with statsmodels 
"""
import numpy as np#Used for high level Data Indexing processes
import statsmodels.api as sm#descriptive statistics and estimation of statistical models

# Generate some sample data
num_periods = 9
all_values = np.array([np.random.random(8) \
                       for i in range(num_periods)])

# Filter the data
y_values = all_values[:, 0] # First column values as Y
x_values = all_values[:, 1:] # All other values as X
x_values = sm.add_constant(x_values) # Include the intercept
results = sm.OLS(y_values, x_values).fit() # Regress and fit the model

In [23]:
print(results.summary())#summaries of the result

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.981
Model:                            OLS   Adj. R-squared:                  0.846
Method:                 Least Squares   F-statistic:                     7.262
Date:                Mon, 28 Jun 2021   Prob (F-statistic):              0.278
Time:                        03:41:00   Log-Likelihood:                 17.541
No. Observations:                   9   AIC:                            -19.08
Df Residuals:                       1   BIC:                            -17.50
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          1.0443      0.297      3.521      0.1

C:\Users\Harleen\anaconda3\lib\site-packages\scipy\stats\stats.py:1603: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=9
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [24]:
print(results.params)#display coefficients of interest

[ 1.04433061 -0.31980853 -0.46105303  0.62000456 -0.49229857 -1.30010007
  1.13888814 -0.34856107]


# Linear optimization

## A maximization example with linear programming

In [25]:
""" 
A linear optimization problem with 2 variables
"""
import pulp#pulp is to describe optimisation problems as mathematical models.
#creates LpVariable to hold the variables of objective function
x = pulp.LpVariable('x', lowBound=0)# Create a variable x >= 0
y = pulp.LpVariable('y', lowBound=0)#Create a variable y>=0

problem = pulp.LpProblem( 
    'A simple maximization objective', 
    pulp.LpMaximize)# Create a LP Maximization problem
problem += 3*x + 2*y, 'The objective function'
# Constraints:
problem += 2*x + y <= 100, '1st constraint'
problem += x + y <= 80, '2nd constraint'
problem += x <= 40, '3rd constraint'
problem.solve()#begin performing linear optimization

C:\Users\Harleen\anaconda3\lib\site-packages\pulp\pulp.py:1199: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

In [26]:
print("Maximization Results:")
for variable in problem.variables():
    print(variable.name, '=', variable.varValue)#each variable value is printed
    #the result shows that maximum value of 180 is possible when the value of x is 20 and value of y is 60

Maximization Results:
x = 20.0
y = 60.0


## A minimization example with integer programming

In [27]:
""" 
An example of implementing an integer 
programming model with binary conditions 
"""
import pulp#pulp is to describe optimisation problems as mathematical models.

dealers = ['X', 'Y', 'Z']#dealers variable contains dictionary identifier which will be used as to rference both dictionary and list
variable_costs = {'X': 500, 'Y': 350, 'Z': 450}#contract cost
fixed_costs = {'X': 4000, 'Y': 2000, 'Z': 6000}#fixed fees charge of each dealer

# Define PulP LPvariables to solve:
quantities = pulp.LpVariable.dicts('quantity', 
                                   dealers, #dealers used as reference
                                   lowBound=0,#lower bound 0 for quantities variable
                                   cat=pulp.LpInteger)#quantities values treated as integer objects
is_orders = pulp.LpVariable.dicts('orders', 
                                  dealers,#dealers used as reference
                                  cat=pulp.LpBinary)#is_orders values treated as binary objects
 #the value of is_orders indicate whether one should enter into transaction with a dealer

In [28]:
"""
This is an example of implementing an integer programming model
with binary variables the wrong way.
"""
# Initialize the model with constraints
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
model += sum([(variable_costs[i] * \
               quantities[i] + \
               fixed_costs[i])*is_orders[i] \
              for i in dealers]), 'Minimize portfolio cost'#this is where things goes wrong 
                                    #we are trying to multiply two variables quantities[i] and is_order[i] 
                                  #mulptiplication of two variable expressions not valid
model += sum([quantities[i] for i in dealers]) == 150\
    , 'Total contracts required'
model += 30 <= quantities['X'] <= 100\
    , 'Boundary of total volume of X'
model += 30 <= quantities['Y'] <= 90\
    , 'Boundary of total volume of Y'
model += 30 <= quantities['Z'] <= 70\
    , 'Boundary of total volume of Z'
model.solve()  # You will get an error running this code!

TypeError: Non-constant expressions cannot be multiplied

In [29]:
"""
This is an example of implementing an 
IP model with binary variables the correct way.
"""
# Initialize the model with constarints
#the above problem can be solved with some intelligent mainpulation
model = pulp.LpProblem('A cost minimization problem',
                       pulp.LpMinimize)
#now we multiply is_orders only with fixed_cost and not with quantities
#instead we write quantities as a function of is_orders 
model += sum(
    [variable_costs[i]*quantities[i] + \
         fixed_costs[i]*is_orders[i] for i in dealers])\
    , 'Minimize portfolio cost'
model += sum([quantities[i] for i in dealers]) == 150\
    ,  'Total contracts required'
model += is_orders['X']*30 <= quantities['X'] <= \
    is_orders['X']*100, 'Boundary of total volume of X'
model += is_orders['Y']*30 <= quantities['Y'] <= \
    is_orders['Y']*90, 'Boundary of total volume of Y'
model += is_orders['Z']*30 <= quantities['Z'] <= \
    is_orders['Z']*70, 'Boundary of total volume of Z'
model.solve()#your code will be executed successfully

1

In [30]:
print('Minimization Results:')
for variable in model.variables():
    print(variable, '=', variable.varValue)#print the variables used in computation 
             #values of is_orders corresponding to dealers reference and value of quantities corresponding to dealers reference

print('Total cost:',  pulp.value(model.objective))#print the Minimized cost of above code

Minimization Results:
orders_X = 0.0
orders_Y = 1.0
orders_Z = 1.0
quantity_X = 0.0
quantity_Y = 90.0
quantity_Z = 60.0
Total cost: 66500.0


# Solving linear equations using matrices

In [1]:
""" 
Linear algebra with NumPy matrices 
"""
import numpy as np#Numpy provides fast mathematical function processing

A = np.array([[2, 1, 1],[1, 3, 2],[1, 0, 0]])# initialized with 3x3 square matrix(2-D array)
B = np.array([4, 5, 6])#input data of right hand side

Use the `linalg.solve` function of NumPy to solve a system of linear scalar
equations:

In [2]:
print(np.linalg.solve(A, B))#solve the equation AX=B using linalg.solve numpy library

[  6.  15. -23.]


# The LU decomposition

In [43]:
 """ 
LU decomposition with SciPy 
"""    
"""
LU decomposition decomposes matrix into a lower triangular matrix L and upper triangular matrix U
a lower triangular matrix has values in lower triangle with remaining upper triangle having value 0
an upper triangular matrix  has values in upper triangle with remaining lower triangle having value 0
A=LU 
a b c     l11  0     0          u11 u12 u13
d e f   = l21  l22   0      x   u21 u22  0
g h i     l31  l32  l33         u31  0   0
"""    
import numpy as np#Used for high level Data Indexing processes
import scipy.linalg as linalg#library of NumPy to solve a system of linear scalar equations


# Define A and B
A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0.]])#3x 3 square matrix initialized which will be decomposed into lower triangular and upper triangular matrix
B = np.array([4., 5., 6.])#input data for right hand side

# Perform LU decompositiopn
LU = linalg.lu_factor(A)#gives LU variable as the pivoted LU decomposition of matrix  A 
x = linalg.lu_solve(LU, B)#pivoted LU decomposition soved with matrix B

In [44]:
print(x)#print the solution output

[  6.  15. -23.]


In [45]:
import scipy#Algorithm library used for high level mathematics, plotting, and statistics

P, L, U = scipy.linalg.lu(A)#lu function of linalg library used for LU decomposition
#lu decomposes matrix into three variables permautation matrix,lower triangular matrix,and upper triangular matrix 
print('P=\n', P)#permutation matrix
print('L=\n', L)#lower traingular matrix
print('U=\n', U)#upper triangular matrix

P=
 [[1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]
L=
 [[ 1.   0.   0. ]
 [ 0.5  1.   0. ]
 [ 0.5 -0.2  1. ]]
U=
 [[ 2.   1.   1. ]
 [ 0.   2.5  1.5]
 [ 0.   0.  -0.2]]


# The Cholesky decomposition

In [46]:
""" 
Cholesky decomposition with NumPy 
"""
"""
Cholesky Decomposition is twice as efficient as LU decomposition.It occupies less memory than LU decomposition.

The necessary condition for Cholesky decomposition is that matrix should be hermitian  

"conjugate of lower triangular matrix=conjugate transpose of lower triangualr matrix".

Matrix is decomposed into L(lower triangular Matrix) and L*( conjugate transpose of lower triangular Matrix)
A=L.L*



"""
import numpy as np#Numpy provides fast mathematical function processing

A = np.array([
    [10., -1., 2., 0.],
    [-1., 11., -1., 3.],
    [2., -1., 10., -1.],
    [0., 3., -1., 8.]])#4x4 hermitian square matrix inatialized 
B = np.array([6., 25., -11., 15.])#input data of right hand side

L = np.linalg.cholesky(A)#cholesky function computes the lower triangular matrix

In [47]:
print(L)#print the lower triangular matrix

[[ 3.16227766  0.          0.          0.        ]
 [-0.31622777  3.3015148   0.          0.        ]
 [ 0.63245553 -0.24231301  3.08889696  0.        ]
 [ 0.          0.9086738  -0.25245792  2.6665665 ]]


In [48]:
print(np.dot(L, L.T.conj()))  # A=L.L* 

[[10. -1.  2.  0.]
 [-1. 11. -1.  3.]
 [ 2. -1. 10. -1.]
 [ 0.  3. -1.  8.]]


In [49]:
y = np.linalg.solve(L, B)  # L.L*.x=B; When L*.x=y, then L.y=B

In [50]:
x = np.linalg.solve(L.T.conj(), y)  # x=L*'.y

In [51]:
print(x) #print  the value of computed x

[ 1.  2. -1.  1.]


In [52]:
print(np.mat(A) * np.mat(x).T)  # B=Ax

[[  6.]
 [ 25.]
 [-11.]
 [ 15.]]


# The QR decomposition

In [53]:
""" 
QR decomposition with scipy 
"""
import numpy as np#Numpy provides fast mathematical function processing
import scipy.linalg as linalg#library of NumPy to solve a system of linear scalar equations


A = np.array([
    [2., 1., 1.],
    [1., 3., 2.],
    [1., 0., 0]])#3x 3 square matrix initialized which will be decomposed into lower triangular and upper triangular matrix
B = np.array([4., 5., 6.])#input data of right hand side

Q, R = scipy.linalg.qr(A)  # QR decomposition
y = np.dot(Q.T, B)  # Let y=Q'.B
x = scipy.linalg.solve(R, y)  # Solve Rx=y

In [54]:
print(x)

[  6.  15. -23.]


# Solving with other matrix algebra methods

## The Jacobi method

In [55]:
"""
Solve Ax=B with the Jacobi method 
"""
import numpy as np

def jacobi(A, B, n, tol=1e-10):
    # Initializes x with zeroes with same shape and type as B
    x = np.zeros_like(B)

    for iter_count in range(n):
        x_new = np.zeros_like(x)
        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (B[i] - s1 - s2) / A[i, i]

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [56]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 25

In [57]:
x = jacobi(A, B, n)
print('x', '=', x)

x = [ 1.  2. -1.  1.]


# The Gauss-Seidel method

In [58]:
""" 
Solve Ax=B with the Gauss-Seidel method 
"""
import numpy as np


def gauss(A, B, n, tol=1e-10):
    L = np.tril(A)  # returns the lower triangular matrix of A
    U = A-L  # decompose A = L + U
    L_inv = np.linalg.inv(L)
    x = np.zeros_like(B)

    for i in range(n):
        Ux = np.dot(U, x)
        x_new = np.dot(L_inv, B - Ux)

        if np.allclose(x, x_new, tol):
            break

        x = x_new

    return x

In [59]:
A = np.array([
    [10., -1., 2., 0.], 
    [-1., 11., -1., 3.], 
    [2., -1., 10., -1.], 
    [0.0, 3., -1., 8.]])
B = np.array([6., 25., -11., 15.])
n = 100
x = gauss(A, B, n)

In [60]:
print('x', '=', x)

x = [ 1.  2. -1.  1.]
